# Протоколы аутентификации с нулевым разглашением

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел
from Crypto.Util import number

# КГПСЧ
from Crypto.Random import random

# Немного математики
import gmpy2

# Для генерации идентификаторов клиентов
import uuid

---

In [4]:
def randint(a, b) -> gmpy2.mpz:
    """
    Генерация случайного числа с помощью КГПСЧ с поддержкой gmpy2
    """
    
    native_a = int(a)
    native_b = int(b)
    
    return gmpy2.mpz(random.randint(native_a, native_b))

----

## Протокол Фиата-Шамира
![PAP Protocol](./images/Fiat-Shamir_2.png)

In [5]:
class FiatShamirCA(object):
    PRIME_BITS = 1024
    
    def __init__(self):
        """
        Инициализация доверенного центра.
        Генерируются два простых числа и их произведение.
        """
        
        self._db = {}
        
        self._p = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._q = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._modulus = gmpy2.mul(self._p, self._q)
        
        trace('[FiatShamirCA]', f'Modulus = {self._modulus}')
        
        
    def register_client(self, client_id: uuid.UUID, client_public_key: gmpy2.mpz) -> None:
        """
        Регистрация клиента. На вход принимает идентификатор клиента и его открытый ключ.
        Ничего не возвращает.
        Выбрасывает исключение, если клиент уже существует.
        """
        
        trace('[FiatShamirCA]', f'''Attempting to register client {client_id} 
        with public key {client_public_key}''')
        
        if client_id in self._db:
            trace('[FiatShamirCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Теперь сохраняю открытый ключ
        #
        
        self._db[client_id] = client_public_key
        trace('[FiatShamirCA]', 'Client registered successfully')
        
        
    def get_public_key(self, client_id: uuid.UUID) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента. Получает на вход идентификатор клиента.
        Возвращает открытый ключ.
        Выбрасывает исключение, если пользователь отсутствует.
        """
        
        return self._db[client_id]
       
        
    @property
    def modulus(self) -> gmpy2.mpz:
        """
        Возвращает модуль.
        """
        
        return self._modulus

In [6]:
class FiatShamirClient(object):
    def __init__(self, modulus: gmpy2.mpz, *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz] = None):
        """
        Инициализация клиента с выработкой ключевой пары и идентификатора.
        Последний параметр требуются для демонстрации неудачной аутентификации.
        """
        
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        self._r = None
        
        self._private_key = FiatShamirClient._generate_coprime(modulus)
        self._public_key = gmpy2.powmod(self._private_key, 2, modulus) if fake_data is None else fake_data[1]
        
        trace('[FiatShamirClient]', f'Client {self._id}')
        trace('[FiatShamirClient]', f'Public key = {self._public_key}')
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора.
        """
        
        return self._id
    
    
    def get_x(self, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение случайного значения, вычисляемого по формуле:
                    x = r ** 2 mod n,
        где n - модуль, r - случайное целое число из отрезка [1, n - 1].
        Значение r сохраняется.
        """
        
        self._r = randint(1, modulus - 1)
        return gmpy2.powmod(self._r, 2, modulus)
    
    
    def get_y(self, e: gmpy2.mpz, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение числа, вычисляемого по формуле:
                    y = r * s ** e mod n,
        где n - модуль, s - закрытый ключ пользователя, r - сгенерированное 
        функцией get_x внутреннее значение, e - целое число из отрезка [0, 1],
        пришедшее с сервера.
        """
        
        product = gmpy2.mul(self._r, gmpy2.powmod(self._private_key, e, modulus))
        return gmpy2.t_mod(product, modulus)
        
        
    @staticmethod
    def _generate_coprime(modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Генерация случайного числа из отрезка [1, n - 1], взаимно
        простого с n, где n - модуль.
        """
        
        #
        # Генерирую число до тех пор, пока оно не станет взаимно простым с модулем
        #
        
        result = randint(1, modulus - 1)
        while gmpy2.gcd(result, modulus) != 1:
            result = randint(1, modulus - 1)
            
        return result

In [7]:
class FiatShamirVerifier(object):
    def __init__(self, iterations: int):
        """
        Инициализация проверяющего. Задает коичество итераций проверки.
        """
        
        self._iterations = iterations
        
        
    def authenticate(self, client: FiatShamirClient, ca: FiatShamirCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        trace('[FiatShamirVerifier]', f'Attempt to authenticate client {client.identifier}')
        trace('[FiatShamirVerifier]', f'Number of iterations = {self._iterations}')
        
        is_successful = True
        
        for i in range(self._iterations):
            x = client.get_x(ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {x = }')
            
            e = randint(0, 1)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {e = }')
            
            y = client.get_y(e, ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {y = }')
            
            pk = ca.get_public_key(client.identifier)
            product = gmpy2.mul(x, gmpy2.powmod(pk, e, ca.modulus))
            
            successful_iteration = gmpy2.powmod(y, 2, ca.modulus) == gmpy2.t_mod(product, ca.modulus)
            is_successful &= successful_iteration
            
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] success: {successful_iteration}')
            
            if not successful_iteration:
                break
            
        print(f'Authentication successful: {is_successful}')
        return is_successful

----

In [8]:
# Создаю доверенный центр
ca = FiatShamirCA()

[TRACE] [FiatShamirCA] Modulus = 18207976070617177277407690177393897010243132803040229445436794775715262669379905375947258668850804314846209443566562624630767627816806569363266013369888831799940469525666232452524191632910605777065928727701862310663899167294914222613543987559344492316027909498733972722666273238874815477898203574463147202375376133554476031585774312107015064200969220569825311656754169634068950774096174048277202439151076881853926289166121641143443215679602154715926090998918476285641371273400511737497751538228348630617504240262730732997009375011913407909168062405398812320260756520648802984315747087512725616289033131776202513590821


In [9]:
# Создаю двух пользователей:
# - Алису - валидного пользователя
# - Еву - пользователя, представляющегося Алисой. но не знающего закрытого ключа Алисы
alice = FiatShamirClient(ca.modulus)
eve = FiatShamirClient(ca.modulus, 
                       fake_data=(alice.identifier, alice.public_key))

[TRACE] [FiatShamirClient] Client 94a12dd0-7fb8-4210-af85-73ad973d8f63
[TRACE] [FiatShamirClient] Public key = 11196711433859076989375203987093521116794759433524183722390367706760451162390673102369319257561882320098041366376625249477047890715172815280730094248601637903098699438775217207743973483418594707080953920303619520563392398093505104788444733923208954598471737664532311841541389493909618980392188712203506930750019445489733679363430219840564223407266515885287614446612611260046470295406952697650912356076878939497610404302605379607122732291145320501563461047751645318489335672517502249516405775721826057657978301874698235144583603525601032553383460932919880435777137290127792514237625534438429707789081623709711983357575
[TRACE] [FiatShamirClient] Client 94a12dd0-7fb8-4210-af85-73ad973d8f63
[TRACE] [FiatShamirClient] Public key = 1119671143385907698937520398709352111679475943352418372239036770676045116239067310236931925756188232009804136637662524947704789071517281528073009424860163790309

In [10]:
# Регистрирую валидного пользователя
ca.register_client(alice.identifier, alice.public_key)

[TRACE] [FiatShamirCA] Attempting to register client 94a12dd0-7fb8-4210-af85-73ad973d8f63 
        with public key 11196711433859076989375203987093521116794759433524183722390367706760451162390673102369319257561882320098041366376625249477047890715172815280730094248601637903098699438775217207743973483418594707080953920303619520563392398093505104788444733923208954598471737664532311841541389493909618980392188712203506930750019445489733679363430219840564223407266515885287614446612611260046470295406952697650912356076878939497610404302605379607122732291145320501563461047751645318489335672517502249516405775721826057657978301874698235144583603525601032553383460932919880435777137290127792514237625534438429707789081623709711983357575
[TRACE] [FiatShamirCA] Client registered successfully


In [11]:
# Для простоты будет 3 итерации
ITERATIONS = 3

# Создаю проверяющую сторону
verifier = FiatShamirVerifier(ITERATIONS)

In [12]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client 94a12dd0-7fb8-4210-af85-73ad973d8f63
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(12386159688413552123595376413758051940402982346905381347928305253238218181297080013755943563844640508446280449228044512603482466696058318290237414252521600359449807187566013642367199496490058087812565306173101610931177716023861483433295387021398731186080949073373934249663288233131578614580930054912771657608723494896092804010168504067979992518410204685910333905913068964463238094432657227593516665950501892868895370092827965112622240138025548218015893299221048281950790972402162285748573913554955810737315128543400358631002702306040844001412368222003993343236948238413104001897202040944547523430100118373879218402325)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(1)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(81163691220730731138884576048318160694875172760900934152543730087435549313

True

In [13]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client 94a12dd0-7fb8-4210-af85-73ad973d8f63
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(11855243617007876626663822687137770353518997228762584006126895582134015397210187198450120089801938702216319039491018353328105737669222174131364429964813427198333684093966200391837404777577489450354989840855289045382359217447482003979909661036896645305087435683989272639476468103553231752035984217236139776957199341318645623093020055635275178005324430252346262511978737199812308006077709800822155037932591947206477750973499810831926408087587103283875627658081927446562671248838732733637370693890510527708566238671760449968383785204498859012637644767224446039686374840627469042731304754143366307625250344276161309474021)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(0)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(15513833443142872322651094888210581005861297238440748908390720672308663037

False